In [1]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [2]:
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os

In [3]:
# colab 세션에 파일을 업로드해도 되고, google-drive mount 후 해당 파일들이 있는 폴더로 옮긴 후 진행하셔도 됩니다.
df = pd.read_csv('final_train_1.txt', sep='\t', names=['label', 'comment'])
df_val = pd.read_csv('final_validation_1.txt', sep='\t', names=['label', 'comment'])
df_test = pd.read_csv('final_test_1.txt', sep='\t', names=['label', 'comment'])

In [4]:
df.head()

,label,comment
0,0,바가지를너무씌워서다시는안가기로했읍니다.지금이어느땐데가격표도없구부르는데로받더라구요.한...
1,0,날씨와 코로나때문에 아쉬웠습니다. 놀이기구 타는 시간도 너무 짧아서 타다만느낌이었어요ㅜㅜ
2,0,망원한강공원가는 출입구가 현재는 하나인데 그마저도 너무 협소 사람 둘이 서서 간신히...
3,0,탈수있는게 별로 없어요
4,0,매표소 직원분들이 온라인 예매에 대해 잘 모르시는 것 같습니다 당일온라인예매는 당일...


In [5]:
comments = df.comment

In [6]:
len(df)

8400

In [7]:
# train 데이터를 기준으로 bpe 모델을 훈련해 줍니다.
# 기타 토크나이저를 사용해도 무방합니다.
vocab_filename = "travel_encoder"

if not os.path.isfile(vocab_filename+".subwords"):
    encoder = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
        comments, target_vocab_size = 20000)

    encoder.save_to_file(vocab_filename)
else:
    encoder = tfds.deprecated.text.SubwordTextEncoder.load_from_file(vocab_filename)


In [8]:
encoder.encode(comments[0])

[1850,
 581,
 226,
 8352,
 8526,
 4237,
 1168,
 9385,
 1388,
 2662,
 14619,
 2968,
 1948,
 1367,
 9000,
 218,
 9690,
 5471,
 219,
 12518,
 218,
 258,
 2448,
 515,
 14619,
 9989,
 1104,
 5807,
 98,
 11185,
 218,
 2376,
 5807,
 181,
 121,
 1927,
 7359,
 3238,
 34,
 14619,
 13498,
 4638,
 3389,
 1212,
 5734,
 195,
 2247,
 515,
 14619,
 227,
 13314,
 6256,
 1808,
 2602,
 5807,
 2190,
 581,
 12985,
 73,
 10740,
 7673,
 4399,
 2582,
 313]

In [9]:
comments[0]

'바가지를너무씌워서다시는안가기로했읍니다.지금이어느땐데가격표도없구부르는데로받더라구요.한국사람한테도이러는데외국인한테는어찌할려는지요.다른곳에서3만원파는걸10만원에달라더라구요.그동안외국에서오는친구들한테소개를많이했는데이젠가지말라고하려구요'

In [10]:
# encoder로 commnet를 encode해준 뒤, 각 df별로 "comment_encoded"라는 column에 추가해 줍니다.
df["comment_encoded"] = df.comment.apply(encoder.encode)
df_val["comment_encoded"] = df_val.comment.apply(encoder.encode)
df_test["comment_encoded"] = df_test.comment.apply(encoder.encode)

In [11]:
# encoder에 대한 정보입니다.
vocab_size = encoder.vocab_size
print(vocab_size)
encoder.subwords[:30]

14829


['. ',
 '너무_',
 ', ',
 '도_',
 '이_',
 '는_',
 '을_',
 '..',
 '수_',
 '에_',
 '고_',
 '은_',
 '많이_',
 '가_',
 '...',
 '.. ',
 '사람이_',
 '좀_',
 '서_',
 '의_',
 '있는_',
 '좋아요',
 '더_',
 '좋은_',
 '잘_',
 '를_',
 '다_',
 '다',
 '한_',
 '만_']

In [12]:
df.head()

,label,comment,comment_encoded
0,0,바가지를너무씌워서다시는안가기로했읍니다.지금이어느땐데가격표도없구부르는데로받더라구요.한...,"[1850, 581, 226, 8352, 8526, 4237, 1168, 9385,..."
1,0,날씨와 코로나때문에 아쉬웠습니다. 놀이기구 타는 시간도 너무 짧아서 타다만느낌이었어요ㅜㅜ,"[1203, 66, 1286, 629, 1, 163, 855, 2416, 2, 14..."
2,0,망원한강공원가는 출입구가 현재는 하나인데 그마저도 너무 협소 사람 둘이 서서 간신히...,"[6762, 3374, 224, 5942, 14, 7306, 10038, 100, ..."
3,0,탈수있는게 별로 없어요,"[7507, 67, 189]"
4,0,매표소 직원분들이 온라인 예매에 대해 잘 모르시는 것 같습니다 당일온라인예매는 당일...,"[1857, 2323, 2674, 864, 10, 1362, 25, 12863, 6..."


In [13]:
# train, test별로 input과 target을 분리해 줍니다.
# 0번 column: label, 2번 column: encoded comment
x_train, y_train = df.iloc[:,2], df.iloc[:,0]
x_val, y_val = df_val.iloc[:,2], df_val.iloc[:,0]
x_test, y_test = df_test.iloc[:,2], df_test.iloc[:,0]

In [14]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [15]:
# one-hot vector로 치환
y_train_categorical = tf.keras.utils.to_categorical(y_train)
y_val_categorical = tf.keras.utils.to_categorical(y_val)
y_test_categorical = tf.keras.utils.to_categorical(y_test)
y_train_categorical

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [16]:
# 훈련에 들어갈 수 있게 데이터를 변환해 줍니다.

from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 200
x_train = pad_sequences(x_train, padding="post", maxlen=maxlen)
x_val = pad_sequences(x_val, padding="post", maxlen=maxlen)
x_test = pad_sequences(x_test, padding="post", maxlen=maxlen)

In [17]:
x_train.shape

(8400, 200)

In [18]:
x_val.shape

(1800, 200)

In [19]:
x_test.shape

(1800, 200)

In [20]:
# test class 비중 파악
print(sum(y_test == 0)) # 나쁨
print(sum(y_test == 1)) # 좋음
print(sum(y_test == 2)) # 보통

600
600
600


In [21]:
sum(y_test == 1) / len(y_test) # '좋음'만 할 경우의 정확도

0.3333333333333333

## DNN 기반의 classifier

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen))
model.add(GlobalAveragePooling1D())
model.add(Dense(3))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['acc'])

In [43]:
history = model.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=10)

Epoch 1/10
263/263 [==============================] - 3s 11ms/step - loss: 1.0984 - acc: 0.3436 - val_loss: 1.0971 - val_acc: 0.3483
Epoch 2/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0962 - acc: 0.3557 - val_loss: 1.0953 - val_acc: 0.3539
Epoch 3/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0944 - acc: 0.3720 - val_loss: 1.0936 - val_acc: 0.3950
Epoch 4/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0923 - acc: 0.4188 - val_loss: 1.0916 - val_acc: 0.4183
Epoch 5/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0899 - acc: 0.4221 - val_loss: 1.0893 - val_acc: 0.4567
Epoch 6/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0872 - acc: 0.4731 - val_loss: 1.0868 - val_acc: 0.5128
Epoch 7/10
263/263 [==============================] - 3s 10ms/step - loss: 1.0840 - acc: 0.4960 - val_loss: 1.0838 - val_acc: 0.5250
Epoch 8/10
263/263 [==============================] - 3s 10ms/step - 

In [44]:
model.evaluate(x_test, y_test_categorical)

57/57 [==============================] - 0s 1ms/step - loss: 1.0739 - acc: 0.5528


[1.0738519430160522, 0.5527777671813965]

## Conv1D 기반의 classifier(TextCNN)

In [36]:
model2 = tf.keras.Sequential()
model2.add(Embedding(vocab_size, 128, input_length=maxlen))
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D(5))
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D())
model2.add(tf.keras.layers.Conv1D(32, 7, activation='relu'))
model2.add(tf.keras.layers.MaxPooling1D())
model2.add(GlobalAveragePooling1D())
model2.add(Dense(3))

model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['acc'])

In [37]:
history2 = model2.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=10)

Epoch 1/10
263/263 [==============================] - 6s 22ms/step - loss: 1.0962 - acc: 0.3752 - val_loss: 1.0887 - val_acc: 0.4100
Epoch 2/10
263/263 [==============================] - 5s 20ms/step - loss: 1.0787 - acc: 0.4154 - val_loss: 1.0695 - val_acc: 0.3789
Epoch 3/10
263/263 [==============================] - 5s 20ms/step - loss: 1.0419 - acc: 0.4486 - val_loss: 1.0166 - val_acc: 0.4817
Epoch 4/10
263/263 [==============================] - 5s 20ms/step - loss: 0.9375 - acc: 0.5189 - val_loss: 0.9124 - val_acc: 0.5333
Epoch 5/10
263/263 [==============================] - 5s 20ms/step - loss: 0.7808 - acc: 0.6098 - val_loss: 0.8405 - val_acc: 0.5856
Epoch 6/10
263/263 [==============================] - 5s 20ms/step - loss: 0.6639 - acc: 0.6550 - val_loss: 0.8268 - val_acc: 0.6094
Epoch 7/10
263/263 [==============================] - 5s 20ms/step - loss: 0.5909 - acc: 0.6865 - val_loss: 0.8337 - val_acc: 0.6039
Epoch 8/10
263/263 [==============================] - 5s 20ms/step - 

In [38]:
model2.evaluate(x_test, y_test_categorical)

57/57 [==============================] - 0s 4ms/step - loss: 1.0132 - acc: 0.5911


[1.013167142868042, 0.5911111235618591]

## LSTM 기반의 classifier

In [45]:
model3 = tf.keras.Sequential()
model3.add(Embedding(vocab_size, 64, input_length=maxlen))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, activation='relu')))
model3.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, activation='relu')))
model3.add(tf.keras.layers.Dense(8, activation='relu'))
model3.add(tf.keras.layers.Dense(3))

model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['acc'])

history3 = model3.fit(x=x_train, y=y_train_categorical, validation_data=(x_val, y_val_categorical), epochs=10)

Epoch 1/10
263/263 [==============================] - 111s 407ms/step - loss: 1.0961 - acc: 0.3877 - val_loss: 1.0899 - val_acc: 0.4233
Epoch 2/10
263/263 [==============================] - 103s 392ms/step - loss: 1.2650 - acc: 0.4592 - val_loss: 1.1028 - val_acc: 0.4656
Epoch 3/10
263/263 [==============================] - 102s 386ms/step - loss: 1.0406 - acc: 0.4770 - val_loss: 763.0280 - val_acc: 0.4800
Epoch 4/10
263/263 [==============================] - 102s 387ms/step - loss: 1.0233 - acc: 0.4835 - val_loss: 1.1745 - val_acc: 0.4861
Epoch 5/10
263/263 [==============================] - 106s 404ms/step - loss: 1.0062 - acc: 0.5212 - val_loss: 1.0010 - val_acc: 0.5033
Epoch 6/10
263/263 [==============================] - 100s 381ms/step - loss: 0.9454 - acc: 0.5536 - val_loss: 3.7788 - val_acc: 0.5389
Epoch 7/10
263/263 [==============================] - 103s 393ms/step - loss: 1.2932 - acc: 0.5588 - val_loss: 1.2804 - val_acc: 0.4806
Epoch 8/10
263/263 [==========================

In [46]:
model3.evaluate(x_test, y_test_categorical)

57/57 [==============================] - 7s 117ms/step - loss: 1.0027 - acc: 0.5500


[1.0027341842651367, 0.550000011920929]

In [ ]:
encoded_comment = df['comment_encoded'][0]

In [ ]:
encoded_comment

[7567, 27, 1665, 14212, 16197, 20, 146, 16198]

In [ ]:
encoder.decode(encoded_comment)

'동물보다 동물 모형이 더 많아요!'